# Week 22: Risk Management - VaR, CVaR, Stress Testing

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Value at Risk (VaR)**: Parametric, Historical, Monte Carlo
- **Expected Shortfall (CVaR)**: Tail risk measure
- **Stress Testing**: Scenario analysis
- **Risk Limits**: Position sizing and controls

---

## Why Risk Management?

"Risk comes from not knowing what you're doing" - Warren Buffett

- Protect capital
- Regulatory compliance
- Survive to trade another day

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 22: Risk Management")

---

## Part 1: Value at Risk (VaR)

### Definition

VaR answers: "What's the maximum loss at X% confidence over Y days?"

$$VaR_{\alpha} = -\inf\{l : P(L > l) \leq 1-\alpha\}$$

### Methods

1. **Parametric**: Assume normal distribution
2. **Historical**: Use actual historical returns
3. **Monte Carlo**: Simulate scenarios

In [ ]:
# Generate sample returns
n_days = 252 * 5
returns = np.random.normal(0.0004, 0.015, n_days)  # Daily returns

# Add some fat tails (realistic)
returns = np.where(np.random.random(n_days) < 0.05,
                   returns * 3,  # 5% chance of 3x moves
                   returns)

portfolio_value = 1_000_000
confidence = 0.95

def var_parametric(returns, confidence=0.95):
    """Parametric VaR assuming normal distribution"""
    mu = returns.mean()
    sigma = returns.std()
    var = stats.norm.ppf(1 - confidence, mu, sigma)
    return -var

def var_historical(returns, confidence=0.95):
    """Historical VaR from empirical distribution"""
    var = np.percentile(returns, (1 - confidence) * 100)
    return -var

def var_montecarlo(returns, confidence=0.95, n_sims=10000):
    """Monte Carlo VaR with simulation"""
    mu = returns.mean()
    sigma = returns.std()
    simulated = np.random.normal(mu, sigma, n_sims)
    var = np.percentile(simulated, (1 - confidence) * 100)
    return -var

print("Value at Risk (95% Confidence, 1-Day)")
print("="*50)
print(f"Portfolio Value: ${portfolio_value:,}")
print(f"\nMethod          VaR %       VaR $")
print("-"*50)

for name, func in [('Parametric', var_parametric), 
                    ('Historical', var_historical),
                    ('Monte Carlo', var_montecarlo)]:
    var_pct = func(returns, confidence)
    var_dollar = var_pct * portfolio_value
    print(f"{name:<15} {var_pct:.2%}       ${var_dollar:,.0f}")

---

## Part 2: Expected Shortfall (CVaR)

### Why CVaR?

VaR only tells you the threshold, not how bad it can get beyond.

$$CVaR_{\alpha} = E[L | L > VaR_{\alpha}]$$

Average loss in the worst (1-α)% scenarios.

In [ ]:
def cvar_historical(returns, confidence=0.95):
    """Expected Shortfall (CVaR)"""
    var = np.percentile(returns, (1 - confidence) * 100)
    cvar = returns[returns <= var].mean()
    return -cvar

var_95 = var_historical(returns, 0.95)
cvar_95 = cvar_historical(returns, 0.95)

print("VaR vs CVaR Comparison")
print("="*50)
print(f"95% VaR:  {var_95:.2%}  (${var_95 * portfolio_value:,.0f})")
print(f"95% CVaR: {cvar_95:.2%}  (${cvar_95 * portfolio_value:,.0f})")
print(f"\nCVaR is {cvar_95/var_95:.1f}x VaR (captures tail risk)")

In [ ]:
# Visualize VaR and CVaR
fig, ax = plt.subplots(figsize=(12, 5))

# Histogram of returns
n, bins, patches = ax.hist(returns, bins=100, density=True, alpha=0.7, color='steelblue')

# Color the tail
var_threshold = -var_historical(returns, 0.95)
for i, (b, p) in enumerate(zip(bins[:-1], patches)):
    if b < var_threshold:
        p.set_facecolor('red')

ax.axvline(var_threshold, color='red', linestyle='--', linewidth=2, label=f'95% VaR = {-var_threshold:.2%}')
ax.axvline(-cvar_95, color='darkred', linestyle=':', linewidth=2, label=f'95% CVaR = {cvar_95:.2%}')

ax.set_xlabel('Daily Return')
ax.set_ylabel('Density')
ax.set_title('Return Distribution with VaR and CVaR')
ax.legend()
plt.show()

---

## Part 3: Stress Testing

### Types

1. **Historical**: Replay past crises (2008, COVID)
2. **Hypothetical**: Design specific scenarios
3. **Reverse**: "What would cause X% loss?"

In [ ]:
# Define historical stress scenarios
stress_scenarios = {
    '2008 Financial Crisis': {'equity': -0.50, 'bonds': 0.10, 'gold': 0.20},
    'COVID Crash (Mar 2020)': {'equity': -0.35, 'bonds': 0.05, 'gold': -0.05},
    '2022 Rate Hikes': {'equity': -0.20, 'bonds': -0.15, 'gold': -0.05},
    'Black Monday 1987': {'equity': -0.22, 'bonds': 0.02, 'gold': 0.05},
}

# Portfolio allocation
portfolio = {
    'equity': 600_000,
    'bonds': 300_000,
    'gold': 100_000
}

print("Stress Test Results")
print("="*60)
print(f"Portfolio: {portfolio}")
print(f"Total: ${sum(portfolio.values()):,}\n")

for scenario, shocks in stress_scenarios.items():
    loss = sum(portfolio[asset] * shocks[asset] for asset in portfolio)
    loss_pct = loss / sum(portfolio.values())
    print(f"{scenario:30} Loss: ${loss:>12,.0f} ({loss_pct:>6.1%})")

---

## Part 4: Risk Limits & Position Sizing

### Kelly Criterion

Optimal bet size for maximizing long-term growth:

$$f^* = \frac{p \cdot b - q}{b} = \frac{\text{edge}}{\text{odds}}$$

Where:
- $p$: Win probability
- $q = 1-p$: Lose probability
- $b$: Win/loss ratio

In [ ]:
def kelly_fraction(win_prob, win_loss_ratio):
    """Calculate Kelly optimal fraction"""
    return (win_prob * win_loss_ratio - (1 - win_prob)) / win_loss_ratio

def var_based_position(capital, var_limit, expected_volatility):
    """Size position based on VaR limit"""
    # VaR = position * volatility * z-score
    z_95 = 1.645
    max_position = var_limit / (expected_volatility * z_95)
    return max_position

# Example
capital = 1_000_000
win_prob = 0.55  # 55% hit rate
avg_win = 0.02   # 2% average win
avg_loss = 0.015 # 1.5% average loss
win_loss_ratio = avg_win / avg_loss

kelly = kelly_fraction(win_prob, win_loss_ratio)
half_kelly = kelly / 2  # More conservative

print("Position Sizing")
print("="*50)
print(f"Win Rate: {win_prob:.0%}")
print(f"Win/Loss Ratio: {win_loss_ratio:.2f}")
print(f"\nKelly Fraction: {kelly:.1%} (${capital * kelly:,.0f})")
print(f"Half Kelly: {half_kelly:.1%} (${capital * half_kelly:,.0f})")

# VaR-based
var_limit = 20_000  # Max daily VaR of $20k
daily_vol = 0.02    # 2% daily volatility
max_pos = var_based_position(capital, var_limit, daily_vol)
print(f"\nVaR-Based Max Position: ${max_pos:,.0f}")

---

## Interview Questions

### Conceptual
1. What are the limitations of VaR?
2. Why is CVaR considered a "coherent" risk measure?
3. How often do VaR breaches occur vs. expected?

### Technical
1. How do you calculate multi-day VaR?
2. What distribution do you use for VaR in practice?
3. How do you stress test a derivatives portfolio?

### Finance-Specific
1. What's the regulatory requirement for VaR?
2. How do you set risk limits for a trading desk?
3. What's the trade-off in Kelly sizing?

---

## Key Takeaways

| Measure | Pros | Cons |
|---------|------|------|
| VaR | Simple, intuitive | Ignores tail severity |
| CVaR | Captures tail risk | Harder to estimate |
| Stress Test | Scenario-specific | Scenarios may not happen |